In [ ]:
# disable warnings
import warnings
warnings.filterwarnings("ignore")

# basic imports
import torch
import timm
import math
import tfimm
import os
import numpy as np
import datetime
import tensorflow as tf 
import tensorflow_addons as tfa
from functools import partial

from keras.utils import dataset_utils
import matplotlib.pyplot as plt

import baseline_config

In [ ]:
########################################################################################
# MODEL PARAMETERS
########################################################################################
MODEL_INPUT_IMAGE_WIDTH = 256
MODEL_INPUT_IMAGE_HEIGHT = 256
MODEL_INPUT_IMAGE_CHANNELS = 1

In [ ]:
def paths_and_labels_to_dataset(image_paths,labels,num_classes):
    path_ds = tf.data.Dataset.from_tensor_slices(image_paths)
    img_ds = path_ds.map(
        lambda path: tf.io.read_file(path), 
        num_parallel_calls=tf.data.AUTOTUNE
    )
    label_ds = dataset_utils.labels_to_dataset(
        labels, 
        'categorical', 
        num_classes)
    img_ds = tf.data.Dataset.zip((img_ds, label_ds))
    return img_ds

def create_dataset(subset):
    image_paths, labels, class_names = dataset_utils.index_directory(
            baseline_config.dataset_path + subset,
            labels="inferred",
            formats=('.pt'),
            class_names=None,
            shuffle=False,
            seed=42,
            follow_links=False)

    dataset = paths_and_labels_to_dataset(
        image_paths=image_paths,
        labels=labels,
        num_classes=len(class_names))
    
    return dataset, class_names

train_dataset, class_names = create_dataset('TRAIN/')
test_dataset, _            = create_dataset('TEST/')
validation_dataset, _      = create_dataset('VALIDATION/')
print("class names: ", class_names)

In [ ]:
def dataset_transforms(image,label):
  
    # reshape into standard 3 channels
    image = tf.io.parse_tensor(image, tf.float32)
    image = tf.expand_dims(image, -1)
    
    # most pre-trained model expect 3 color channels
    #image = tf.repeat(image, 3, 2)
    
    image = tf.ensure_shape(image, [313, 128, 1])
    image = tf.image.resize(image, (MODEL_INPUT_IMAGE_HEIGHT, MODEL_INPUT_IMAGE_WIDTH), method=tf.image.ResizeMethod.LANCZOS5)
    
    # rescale to range [0,1]
    image = image - tf.reduce_min(image) 
    image = image / (tf.reduce_max(image)+tf.keras.backend.epsilon()) 
    
    return image,label
  
def pre_process_augment_image(image, label, augment_rate):

  p_aug = tf.random.uniform([], 0, 1.0, dtype=tf.float32)
  
  if p_aug<augment_rate:
     
    # central crop
    crop_amt = tf.random.uniform([], 0.90, 1.00, dtype=tf.float32)
    image = tf.image.central_crop(image, crop_amt)
    
    # resize back to image size
    image = tf.image.resize(image, (MODEL_INPUT_IMAGE_HEIGHT, MODEL_INPUT_IMAGE_WIDTH), method=tf.image.ResizeMethod.LANCZOS5)
        
    # Random rotation.
    #angle = tf.random.uniform([], -2.0, 2.0, dtype=tf.float32)
    #image = tfa.image.rotate(image, angle * math.pi / 180.0, interpolation='BILINEAR', fill_mode="nearest")
   
    # Random left-right flip
    image = tf.image.random_flip_left_right(image)
    # Random up-down flip
    #image = tf.image.random_flip_up_down(image) 
    
    # rescale to range [0,1]
    image = image - tf.reduce_min(image) 
    image = image / (tf.reduce_max(image)+tf.keras.backend.epsilon())  
    
  return image, label  

In [ ]:
def apply_rejection_sampling(dataset):
  # add rejection sampling to the training dataset
  MAX_TRAINING_SIZE=16*100

  # create a class name to indes mapper
  class_name_map = {}
  for index, class_name in enumerate(class_names):
      class_name_map[class_name] = index
      
  target_distribution = np.full(range(len(class_names)), 1.0 / len(class_names))
  rejection_sampling = tf.data.Dataset.rejection_resample(lambda x,y: class_name_map[y] , target_distribution, seed=42) 
  return dataset.apply(rejection_sampling)

# code adapted from: https://stackoverflow.com/questions/56068588/tensorflow-tf-data-experimental-rejection-resample-does-not-achieve-target-distr
def test_rejection_sampling(dataset):
  bucket_counts = [0, 0]
  for i in range(100):
      for data in dataset:
          class_id, data_content = data
          bucket_counts[class_id.numpy()] += 1
  print("This is your result counts", bucket_counts,
        "This is your final dist", bucket_counts[0] / np.sum(bucket_counts), bucket_counts[1] / np.sum(bucket_counts))
  
train_dataset_rs = apply_rejection_sampling(train_dataset)
test_rejection_sampling(train_dataset_rs)

In [ ]:
# create all the data pipelines

train_dataset_b = ( 
                  train_dataset       
                  .shuffle(10000)
                  .map(dataset_transforms)                  
                  .cache()
                  #.map(partial(pre_process_augment_image, augment_rate=0.5))
                  .batch(baseline_config.batch_size)
                )

validation_dataset_b = ( 
                  validation_dataset
                  .map(dataset_transforms)
                  .cache()
                  .batch(baseline_config.batch_size)
                )

test_dataset_b = ( 
                  test_dataset
                  .map(dataset_transforms)
                  .cache()
                  .batch(baseline_config.batch_size)
                )

In [ ]:
for item,lbl in train_dataset_b.take(3):
    print(item.shape, lbl.shape)
    print(np.min(item), np.max(item))
    plt.figure(figsize=(5,5))
    plt.imshow(item[0,:,:,0], cmap='afmhot')

In [ ]:
# uncomment to list all possible timm models
tfimm.list_models()

In [7]:
class CustomTimmModel(tf.keras.Model):
    
    def __init__(self, *args, **kwargs):  
        super(CustomTimmModel, self).__init__(*args, **kwargs)
        self.fm = tfimm.create_model("efficientnet_v2_s_in21k", pretrained=True, in_channels=MODEL_INPUT_IMAGE_CHANNELS)
        
        self.flat = tf.keras.layers.Flatten()
        #self.flat = tf.keras.layers.GlobalMaxPooling2D()
        
        dropout=0.70
        
        self.fc1 = tf.keras.layers.Dense(len(class_names)*20, activation=tf.keras.activations.gelu)
        self.do1 = tf.keras.layers.Dropout(dropout)        
        
        self.fc2 = tf.keras.layers.Dense(len(class_names)*20, activation=tf.keras.activations.gelu)
        self.do2 = tf.keras.layers.Dropout(dropout)
        
        self.fc3 = tf.keras.layers.Dense(len(class_names)*20, activation=tf.keras.activations.gelu)
        self.do3 = tf.keras.layers.Dropout(dropout)        
        
        self.fc4 = tf.keras.layers.Dense(len(class_names)*1, activation=tf.keras.activations.linear)
        

    def call(self, inputs, training=False):
        
        x = self.fm.forward_features(inputs) 
        x = self.flat(x)
        
        x = self.fc1(x)
        x = self.do1(x)
        
        x = self.fc2(x)               
        x = self.do2(x)
        
        x = self.fc3(x)               
        x = self.do3(x)        
        
        x = self.fc4(x)
        
        return x
        
# construct the model
model = CustomTimmModel()

# model.fm.trainable=False

# need to tell the model what the input shape is
model.build([None, MODEL_INPUT_IMAGE_HEIGHT, MODEL_INPUT_IMAGE_WIDTH, MODEL_INPUT_IMAGE_CHANNELS])

All PyTorch model weights were used when initializing EfficientNet.
All the weights of EfficientNet were initialized from the PyTorch model.



In [8]:
if not os.path.exists('models/'):
    os.mkdir('models/')
    
# the form_logits means the loss function has the 'softmax' buillt in.  This approach is numerically more stable
# than including the softmax activation on the last layer of the classifier
model.compile(loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True), 
              optimizer=tf.keras.optimizers.RMSprop(learning_rate=2e-5), 
              metrics=["accuracy"],
              )

# tensorboard for visualisation of results
log_dir = "tensorboard_logs/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, 
                                                      histogram_freq=1)

# reduce learning rate to avoid overshooting local minima
lr_reduce_plateau = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', 
                                                      factor=0.5,
                                                      patience=4, 
                                                      verbose=1,
                                                      mode='min',
                                                      cooldown=0, 
                                                      min_lr=1e-8)

# end the training if no improvement for 8 epochs in a row, then restore best model weights
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    min_delta=0,
    patience=8,
    verbose=0,
    mode="min",
    baseline=None,
    restore_best_weights=True,
)

# save the best model as it trains..
mcp_save = tf.keras.callbacks.ModelCheckpoint('models/checkpoint_timm_model.hdf5', 
                                            save_weights_only=True,
                                           save_best_only=True, 
                                           monitor='val_loss', 
                                           mode='min')

# fit the model to the training set
history = model.fit(train_dataset_b, 
          validation_data=validation_dataset_b,
          callbacks=[lr_reduce_plateau, early_stopping, tensorboard_callback, mcp_save],
          epochs=500)

Epoch 1/500
387/387 [==============================] - 108s 208ms/step - loss: 1.6729 - accuracy: 0.2196 - val_loss: 1.5825 - val_accuracy: 0.3255 - lr: 2.0000e-05
Epoch 2/500
387/387 [==============================] - 80s 201ms/step - loss: 1.5838 - accuracy: 0.2672 - val_loss: 1.5532 - val_accuracy: 0.3802 - lr: 2.0000e-05
Epoch 3/500
387/387 [==============================] - 82s 206ms/step - loss: 1.5418 - accuracy: 0.3185 - val_loss: 1.4951 - val_accuracy: 0.4896 - lr: 2.0000e-05
Epoch 4/500
387/387 [==============================] - 81s 204ms/step - loss: 1.4994 - accuracy: 0.3583 - val_loss: 1.4287 - val_accuracy: 0.5573 - lr: 2.0000e-05
Epoch 5/500
387/387 [==============================] - 82s 205ms/step - loss: 1.4475 - accuracy: 0.4158 - val_loss: 1.3658 - val_accuracy: 0.5964 - lr: 2.0000e-05
Epoch 6/500
387/387 [==============================] - 82s 205ms/step - loss: 1.3874 - accuracy: 0.4813 - val_loss: 1.3019 - val_accuracy: 0.6927 - lr: 2.0000e-05
Epoch 7/500
387/387 [

In [10]:
# save the model weights
model.save_weights('models/baseline_timm_model.hdf5', save_format='h5')

# test load the model for inference
test_model = CustomTimmModel()
test_model.build([None, MODEL_INPUT_IMAGE_HEIGHT, MODEL_INPUT_IMAGE_WIDTH, MODEL_INPUT_IMAGE_CHANNELS])
test_model.load_weights('models/baseline_timm_model.hdf5')

All PyTorch model weights were used when initializing EfficientNet.
All the weights of EfficientNet were initialized from the PyTorch model.



In [1]:
def rejection_sampling_experiment():
    import tensorflow as tf
    import numpy as np

    # everything is based on tensorflow 2.0
    tf.random.set_seed(2342)


    def map2label(sample):
        return tf.cast(tf.math.equal(sample, 2), tf.int32)


    np_data = np.array([0, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2])
    target_dist = [0.5, 0.5]
    init_dist = [(np_data.shape[0] - 3) / np_data.shape[0], 3 / np_data.shape[0]]

    dataset = tf.data.Dataset.from_tensor_slices(np_data)
    shuffle = tf.data.experimental.shuffle_and_repeat(1, count=1000)
    rej = tf.data.Dataset.rejection_resample(map2label, target_dist, seed=2342)  # set seed explicitly
    dataset = dataset.apply(shuffle).apply(rej)
    dataset = dataset.take(30)

    bucket_counts = [0, 0]
    for i in range(100):
        for data in dataset:
            class_id, data_content = data
            bucket_counts[class_id.numpy()] += 1

    print("This is your target_dist", target_dist, "This is your initial distribution", init_dist)
    print("This is your result counts", bucket_counts,
        "This is your final dist", bucket_counts[0] / np.sum(bucket_counts), bucket_counts[1] / np.sum(bucket_counts))

Instructions for updating:
Use `tf.data.Dataset.shuffle(buffer_size, seed)` followed by `tf.data.Dataset.repeat(count)`. Static tf.data optimizations will take care of using the fused implementation.
Instructions for updating:
Use `tf.data.Dataset.rejection_resample(...)`.
